# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
ali_cities_df = pd.read_csv("cities_Ali.csv")
ali_cities_df.head()

,City,Country,Latitude,Longitude,Temp,Wind,Pressure,Humidity,Date
0,ribeira grande,PT,38.5167,-28.7000,20.22,14.92,1002.0,83.0,1.637591e+09
1,jamestown,US,42.0970,-79.2353,-0.76,1.79,1013.0,82.0,1.637591e+09
2,oistins,BB,13.0667,-59.5333,29.05,8.23,1013.0,70.0,1.637591e+09
3,ushuaia,AR,-54.8000,-68.3000,0.81,10.29,1004.0,93.0,1.637591e+09
4,arraial do cabo,BR,-22.9661,-42.0278,24.92,5.14,1015.0,50.0,1.637591e+09


In [7]:
ali_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       627 non-null    object 
 1   Country    574 non-null    object 
 2   Latitude   579 non-null    float64
 3   Longitude  579 non-null    float64
 4   Temp       579 non-null    float64
 5   Wind       579 non-null    float64
 6   Pressure   579 non-null    float64
 7   Humidity   579 non-null    float64
 8   Date       579 non-null    float64
dtypes: float64(7), object(2)
memory usage: 44.2+ KB


In [11]:
alicities_df2= ali_cities_df.dropna(how="any")
alicities_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574 entries, 0 to 626
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       574 non-null    object 
 1   Country    574 non-null    object 
 2   Latitude   574 non-null    float64
 3   Longitude  574 non-null    float64
 4   Temp       574 non-null    float64
 5   Wind       574 non-null    float64
 6   Pressure   574 non-null    float64
 7   Humidity   574 non-null    float64
 8   Date       574 non-null    float64
dtypes: float64(7), object(2)
memory usage: 44.8+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# Heatmap of humidity
figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black'}

locations = alicities_df2[["Latitude", "Longitude"]]
humidity = alicities_df2["Humidity"]
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
mask1 = (alicities_df2['Temp'] < 80) & (alicities_df2['Temp'] > 70)
mask2 = alicities_df2["Wind"] < 10
mask3 = alicities_df2.Pressure == 0

ali_sub = alicities_df2.loc[mask1 & mask2 & mask3].reset_index(drop=True)
ali_sub = ali_sub.dropna(how="any")
ali_sub

,City,Country,Latitude,Longitude,Temp,Wind,Pressure,Humidity,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# geocoordinates
target_coordinates = f"{test[0]}, {test[1]}"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

response.url

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
